# Despacho múltiple (*multiple dispatch*)

En este *notebook*, exploramos el despacho múltiple (*multiple dispatch*), que es una característica clave de Julia.

El despacho múltiple es hace que el *software* sea **genérico** y **rápido**.

#### Comenzando con lo conocido

Para entender el despacho múltiple en Julia, comencemos con lo que ya hemos visto.

Podemos declarar funciones en Julia sin darle mucha información sobre los tipos de los argumentos de entrada que la función recibirá:

In [1]:
f(x) = x^2

f (generic function with 1 method)

y después Julia determinará por su cuenta qué tipos de argumentos de entrada tienen sentido y cuáles no:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float32, ::Integer) at math.jl:907
  ^(!Matched::Irrational{:ℯ}, ::Integer) at mathconstants.jl:91
  ^(!Matched::Irrational{:ℯ}, ::Number) at mathconstants.jl:91
  ...

#### Especificando los tipos de nuestros argumentos de entrada

Sin embargo, también tenemos la *opción* de decirle a Julia de forma explícita qué tipos están permitidos en nuestros argumentos de entrada.

Por ejemplo, escribamos una función `foo` que solo tome cadenas como entradas.

In [4]:
foo(x::String, y::String) = println("¡Mis entradas x y y son ambas cadenas!")

foo (generic function with 1 method)

Aquí vemos que para poder restringir el tipo de `x` y `y` a `String`, solo debemos escribir después del nombre del argumento de entrada dos dos puntos (::) y la palabra reservada `String`.

Ahora, veremos que `foo` funciona con cadenas y no funciona con otros tipos de argumentos de entrada.

In [5]:
foo("Hola", "Hi")

¡Mis entradas x y y son ambas cadenas!


In [6]:
foo(3, 4)

LoadError: MethodError: no method matching foo(::Int64, ::Int64)

Para hacer que `foo` trabaje en entradas enteras (de tipo `Int`), solo ponemos `::Int` en nuestros argumentos de entrada cuando declaramos `foo`.

In [7]:
foo(x::Int, y::Int) = println("Mis entradas x y y son ambas enteras")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

Mis entradas x y y son ambas enteras


Ahora, `foo` funciona con enteros. Pero mira que `foo` también aún funciona cuando `x` y `y` son cadenas.

In [9]:
foo("hello", "hi!")

¡Mis entradas x y y son ambas cadenas!


Esto comienza a llegar al corazón del despacho múltiple. Cuando declaramos

```julia
foo(x::Int, y::Int) = println("Mis entradas x y y son ambas enteras")
```

no sobreescribimos o reemplazamos

```julia
foo(x::String, y::String)
```

En su lugar, solo agregamos un ***método*** adicional a la ***función genérica*** llamada `foo`.

Una ***función genérica*** es el concepto abstracto asociado con una operación en particular.

Por ejemplo, la función genérica `+` representa el concepto de adición.

Un ***método*** es una implementación específica de una función genérica para *tipos de argumentos particulares*.

Por ejemplo, `+` tiene métodos que aceptan números de punto flotante, enteros, matrices, etc.

Podemos usar los `métodos` para ver cuántos métodos existen para `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[7]:1
[2] foo(x::String, y::String) in Main at In[4]:1

Nota aparte: ¿cuántos métodos piensas que hay para la suma?

In [13]:
+(2, 3, 5)

10

In [14]:
methods(+)

# 184 methods for generic function "+":
[1] +(x::Float32, y::Float32) in Base at float.jl:400
[2] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight) in Pkg.Resolve at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Pkg\src\Resolve\versionweights.jl:22
[3] +(dt::Dates.DateTime, z::Dates.Month) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:47
[4] +(dt::Dates.DateTime, y::Dates.Year) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:23
[5] +(x::Dates.DateTime, y::Dates.Period) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:76
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:116
[8] +(x::Dates.Date, y::Dates.Day) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:74
[9] +(x::Dates.Date, y::Dates.Week) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:72
[10] +(dt::Dates.Date, z::Dates.Month) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:54
[11] +(dt::Dates.Date, y::Dates.Year) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:27
[12] +(dt::Dates.Date, t::Dates.Time) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:19
[13] +(t::Dates.Time, dt::Dates.Date) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:20
[14] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\arithmetic.jl:78
[15] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[16] +(x::Bool, y::Bool) in Base at bool.jl:96
[17] +(x::Bool) in Base at bool.jl:93
[18] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:103
[19] +(x::Bool, z::Complex) in Base at complex.jl:293
[20] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue) in Pkg.Resolve at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Pkg\src\Resolve\fieldvalues.jl:43
[21] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:399
[22] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:518
[23] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:517
[24] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:516
[25] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:476
[26] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:524
[27] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:530
[28] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[29] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\periods.jl:347
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\periods.jl:345
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\Dates\src\periods.jl:375
[33] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at D:\Programas\Julia 1.5.3\share\julia\stdlib\v1.5\LinearAlgebra\src\uniformscaling.jl:117
[34] +(::Missing, ::Missing) in Base at missing.jl:114
[35] +(::Missing) in Base at missing.jl:100
[36] +(::Missing, ::Number) in Base at missing.jl:115
[37] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:548
[38] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:541
[39] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:535
[40] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:394
[41] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:363
[42] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:370
[43] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:378


Así que ahora podemos llamar `foo` con enteros o cadenas. Cuando llamas `foo` para un conjunto específico de argumentos, Julia inferirá los tipos de las entradas y *despachará* el método apropiado. *Esto* es el despacho múltiple.

El despacho múltiple hace que nuestro código sea genérico y rápido. Nuestro código puede ser genérico y flexible porque podemos escribir código en términos de operaciones abstractas tales como la suma y la multiplicación, en lugar de en términos de implementaciones específicas. Al mismo tiempo, nuestro código corre de forma rápida porque Julia es capaz de llamar métodos eficientes para los tipos relevantes.

Para ver qué métodos es el que está siendo despachado cuando llamamos una función genérica, podemos usar la macro `@which`:

In [15]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[7]:1

Veamos qué sucede cuando usamos `@which` con el operador de suma.

In [18]:
@which 3 + 3.0

+(x::Number, y::Number) in Base at promotion.jl:311

Y podemos continuar agregando otros métodos a nuestra función genérica `foo`. Agreguemos uno que tome el ***tipo abstracto*** `Number`, que incluye los subtipos tales como `Int`, `Float64` y otros objetos pudiera pensar como números:

In [19]:
foo(x::Number, y::Number) = println("Mis entradas x y y son ambas números")

foo (generic function with 3 methods)

Este método para `foo` funcionará en, por ejemplo, números de punto flotante:

In [20]:
foo(3.0, 4.0)

Mis entradas x y y son ambas números


También podemos agregar un método que tome entradas de cualquier tipo:

In [21]:
foo(x, y) = println("Acepto entradas de cualquier tipo")

foo (generic function with 4 methods)

Con los métodos que hemos escrito hasta ahora para `foo`, este método será llamado cuando le pasemos cosas que no son números a `foo`:

In [22]:
v = rand(3)
foo(v, v)

Acepto entradas de cualquier tipo


In [23]:
@which foo(v, v)

foo(x, y) in Main at In[21]:1

### Ejercicios

#### 7.1

Extiende la función `foo` al agregar un método que tome únicamente un argumento de entrada que sea de tipo `Bool` y regrese "foo con un booleano"

In [24]:
function foo(x::Bool)::String
  return "foo con un booleano"
end

foo (generic function with 5 methods)

In [27]:
methods(foo)

# 5 methods for generic function "foo":
[1] foo(x::Bool) in Main at In[24]:1
[2] foo(x::Int64, y::Int64) in Main at In[7]:1
[3] foo(x::String, y::String) in Main at In[4]:1
[4] foo(x::Number, y::Number) in Main at In[19]:1
[5] foo(x, y) in Main at In[21]:1

#### 7.2

Verifica que el método despachado al ejecutar 
```julia
foo(true)
```
es el que escribiste.

In [25]:
@which foo(true)

foo(x::Bool) in Main at In[24]:1

In [26]:
@assert foo(true) == "foo con un booleano"